In [1]:
from n2v.models import N2V
import numpy as np
from csbdeep.io import save_tiff_imagej_compatible
from n2v.internals.N2V_DataGenerator import N2V_DataGenerator

2024-04-16 09:06:20.461541: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-16 09:06:20.463938: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-16 09:06:20.497029: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-16 09:06:21.097647: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [9]:

dome_path = './../../ppujol/test_n2v_linux/' # Fill with the path of the dome
name_of_dome = '21-tp1-270.tif' # Fill with the name of the dome
save_dome_path = './../../ppujol/test_n2v_linux/' # Fill with the path to save the dome
name_save_dome = name_of_dome.split('.')[0]+'_denoised.tif'

dims = 'YX'

In [10]:
datagen = N2V_DataGenerator()
img = datagen.load_imgs_from_directory(directory=dome_path, filter=name_of_dome, dims=dims)

print(img[0].shape)

(1, 576, 576, 1)


In [11]:
model_name = '2d_images_from_stack'
basedir = './n2v_2d_black_line/models_test/'
model = N2V(config=None, name=model_name, basedir=basedir)

Loading network weights from 'weights_best.h5'.


In [13]:
def denoise2dSlice(img):
    image = img[:, :, 0]
    # print(image.shape)

    non_zero_cols = np.any(image != 0, axis = 0)
    first_non_zero_col = np.argmax(non_zero_cols)
    last_non_zero_col = len(non_zero_cols) - np.argmax(non_zero_cols[::-1]) - 1
    clean_image = image[:, first_non_zero_col:last_non_zero_col + 1]

    pred = model.predict(clean_image, axes='YX')
    full_pred = np.zeros_like(image)
    full_pred[:, first_non_zero_col:last_non_zero_col + 1] = pred

    return full_pred

def denoise3d(img):
    stack = []
    for i in img:
        stack.append(denoise2dSlice(i))
    stack = np.array(stack)
    return stack

In [15]:
img = img[0]
stack = []

if len(dims) == 2:
    stack = denoise2dSlice(img=img)
if len(dims) == 3:
    stack = denoise3d(img=img)
if len(dims) == 4:
    for i in img:
        stack.append(denoise3d(img=i))
stack = np.array(stack)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 857ms/step


In [16]:
save_tiff_imagej_compatible(str(save_dome_path)+str(name_save_dome), stack, axes=dims)